In [1]:
import pandas as pd
import numpy as np
# NAIBE BAYES
from sklearn.naive_bayes import GaussianNB
# KNN
from sklearn.neighbors import KNeighborsClassifier
# RANDOM FOREST
from sklearn.ensemble import RandomForestClassifier
# LOGISTIC REGRESSION
from sklearn.linear_model import LogisticRegression
# DECISON TREE
from sklearn.tree import DecisionTreeClassifier
# XGBOOST
from xgboost import XGBClassifier
# AdaBoosting Classifier
from sklearn.ensemble import AdaBoostClassifier
# GradientBoosting Classifier
from sklearn.ensemble import GradientBoostingClassifier
# HistGradientBoostingClassifier
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier, StackingClassifier

from sklearn.model_selection import StratifiedKFold, GridSearchCV, cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
import warnings
import time

In [2]:
#讀取CSV檔，將資料放入dataframe
train = pd.read_csv(r'C:\Users\88691\PycharmProjects\hotel_predit\data-question\train.csv', header=None)
test = pd.read_csv(r'C:\Users\88691\PycharmProjects\hotel_predit\data-question\test.csv', header=None)
warnings.filterwarnings("ignore")

In [3]:
#train資料集preprocessing
#顯示資料集
print(train)
#顯示資料筆數
print(train.shape)
#顯示資料統計資訊
print(train.describe())
#顯示欄位屬性
print(train.dtypes)

        0               1                        2              3   \
0       ID  Administrative  Administrative_Duration  Informational   
1     8773               0                        0              0   
2     6709               0                        0              0   
3     1463               9                      301              0   
4     4095               2                13.333333              0   
...    ...             ...                      ...            ...   
8096  3758               0                        0              0   
8097  4437               1                     15.2              2   
8098  7449               0                        0              0   
8099   665               9               183.785714              1   
8100   552               3                    49.75              0   

                          4               5                        6   \
0     Informational_Duration  ProductRelated  ProductRelated_Duration   
1            

In [4]:
#檢查是train否具有遺漏值
total = train.isnull().sum().sort_values(ascending=False)
percent = (train.isnull().sum() / train.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
print(missing_data)

    Total   Percent
6       1  0.000123
16      1  0.000123
15      1  0.000123
14      1  0.000123
13      1  0.000123
12      1  0.000123
18      0  0.000000
1       0  0.000000
2       0  0.000000
3       0  0.000000
4       0  0.000000
5       0  0.000000
9       0  0.000000
7       0  0.000000
8       0  0.000000
17      0  0.000000
10      0  0.000000
11      0  0.000000
0       0  0.000000


In [5]:
# 發現遺漏值欄位為類別變數，且資料筆數佔比遠小於1%，故刪除含有NA的樣本
train_no_na = train.dropna()
print(train_no_na)

        0               1                        2              3   \
0       ID  Administrative  Administrative_Duration  Informational   
1     8773               0                        0              0   
2     6709               0                        0              0   
3     1463               9                      301              0   
4     4095               2                13.333333              0   
...    ...             ...                      ...            ...   
8096  3758               0                        0              0   
8097  4437               1                     15.2              2   
8098  7449               0                        0              0   
8099   665               9               183.785714              1   
8100   552               3                    49.75              0   

                          4               5                        6   \
0     Informational_Duration  ProductRelated  ProductRelated_Duration   
1            

In [6]:
# 刪除第0列(去除欄位名稱)
train_new = train_no_na.drop([0])
print(train_new)

        0  1           2  3     4    5            6         7         8   \
1     8773  0           0  0     0    1            0       0.2       0.2   
2     6709  0           0  0     0    1            0       0.2       0.2   
3     1463  9         301  0     0   38  2621.621429  0.021212  0.044508   
4     4095  2   13.333333  0     0  105  2062.443592  0.012205  0.020276   
5     3346  0           0  0     0   19   220.384849  0.010526  0.024642   
...    ... ..         ... ..   ...  ...          ...       ...       ...   
8096  3758  0           0  0     0    4           81         0      0.05   
8097  4437  1        15.2  2  62.6   84  4941.698611  0.017647  0.038706   
8098  7449  0           0  0     0   25   701.883333         0  0.023333   
8099   665  9  183.785714  1    90   95  3346.501984  0.002118  0.018656   
8100   552  3       49.75  0     0   20   547.400794         0  0.011688   

             9    10 11 12  13 14  15 16 17 18  
1             0  0.4  2  3  11  3   7 

In [7]:
#轉換train資料型態為float
train_new = train_new.astype('float')

In [8]:
#顯示資料統計資訊
print(train_new.describe())

                0            1            2            3            4   \
count  8097.000000  8097.000000  8097.000000  8097.000000  8097.000000   
mean   4502.041250     2.309003    80.802419     0.498209    32.896484   
std    2600.317713     3.311535   179.822259     1.258284   135.234455   
min       0.000000     0.000000     0.000000     0.000000     0.000000   
25%    2241.000000     0.000000     0.000000     0.000000     0.000000   
50%    4506.000000     1.000000     7.000000     0.000000     0.000000   
75%    6761.000000     4.000000    91.888889     0.000000     0.000000   
max    8999.000000    27.000000  3398.750000    24.000000  2549.375000   

                5             6            7            8            9   \
count  8097.000000  8.097000e+03  8097.000000  8097.000000  8097.000000   
mean     31.796468  8.313599e+04     0.020324     0.042678     5.746534   
std      44.966699  6.359881e+06     0.131958     0.047920    17.943550   
min       0.000000  0.000000e+00 

In [9]:
#承上，發現最小值有負數，刪除不合理的樣本(機率應介於0~1之間)
train_new[train_new < 0] = np.nan
train_new = train_new.dropna()
print(train_new.describe())

                0            1            2            3            4   \
count  8095.000000  8095.000000  8095.000000  8095.000000  8095.000000   
mean   4503.152810     2.309450    80.821518     0.498209    32.691023   
std    2599.676938     3.311813   179.840360     1.258415   133.930135   
min       0.000000     0.000000     0.000000     0.000000     0.000000   
25%    2243.500000     0.000000     0.000000     0.000000     0.000000   
50%    4507.000000     1.000000     7.000000     0.000000     0.000000   
75%    6761.500000     4.000000    91.936869     0.000000     0.000000   
max    8999.000000    27.000000  3398.750000    24.000000  2549.375000   

                5             6            7            8            9   \
count  8095.000000  8.095000e+03  8095.000000  8095.000000  8095.000000   
mean     31.795800  8.315607e+04     0.021812     0.042679     5.747954   
std      44.971574  6.360667e+06     0.047767     0.047926    17.945539   
min       0.000000  0.000000e+00 

In [10]:
train_new.to_csv(r'C:\Users\88691\PycharmProjects\hotel_predit\data-question\train_new.csv', index = False)

In [11]:
#分配train資料集
X_train, X_test, y_train, y_test = train_test_split(train_new.iloc[1:, 1:18], train_new.iloc[1:, 18:],
                                                    test_size=0.30, random_state=101)
print(X_train.shape,X_test.shape,y_train.shape,y_test.shape)

(5665, 17) (2429, 17) (5665, 1) (2429, 1)


In [12]:
# 利用StratifiedKFold做交叉驗證，相較於KFold，StratifiedKFold會照比例每個data set中抽取資料作驗證
sk_fold = StratifiedKFold(10, shuffle=True, random_state=42)

In [13]:
g_nb = GaussianNB()
knn = KNeighborsClassifier()  # 參數:n_neighbors(鄰居數:預設為5)、weights(權重,預設為uniform)、leaf_size(葉的大小:預設為30)
ran_for = RandomForestClassifier()
# n_estimators:樹的顆數、max_depth:最大深度，剪枝用，超過全部剪掉。
# min_samples_leaf:搭配max_depth使用，一個節點在分枝後每個子節點都必須包含至少min_samples_leaf個訓練樣本
# bootstrap:重新取樣原有Data產生新的Data，取樣的過程是均勻且可以重複取樣
log_reg = LogisticRegression()  #penalty:懲罰函數(預設L2)、Ｃ:正則強度倒數，預設為1.0、solver:解決器(默認='lbfgs')，saga對所有懲罰都可以使用
tree = DecisionTreeClassifier()
xgb = XGBClassifier()  # https://www.itread01.com/content/1536594984.html 參數詳解
ada_boost = AdaBoostClassifier()  # https://ask.hellobi.com/blog/zhangjunhong0428/12405 參數詳解
grad_boost = GradientBoostingClassifier(n_estimators=100)  # https://www.itread01.com/content/1514358146.html 參數詳解
hist_grad_boost = HistGradientBoostingClassifier()  # https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.HistGradientBoostingClassifier.html

In [14]:
#訓練模型之參數設定
clf = [("Naive Bayes", g_nb, {}), \
       ("K Nearest", knn, {"n_neighbors": [3, 5, 6, 7, 8, 9, 10], "leaf_size": [25, 30, 35]}), \
       ("Random Forest", ran_for,
        {"n_estimators": [10, 50, 100, 200, 400], "max_depth": [3, 10, 20, 40], "random_state": [99],
         "min_samples_leaf": [5, 10, 20, 40, 50], "bootstrap": [False]}), \
       ("Logistic Regression", log_reg, {"penalty": ['l2'], 'max_iter':[10, 20],"C": [100, 10, 1.0, 0.1, 0.01], "solver": ['saga']}), \
 \
       ("Decision Tree", tree, {}), \
       ("XGBoost", xgb,
        {"n_estimators": [200], "max_depth": [3, 4, 5], "learning_rate": [.01, .1, .2], "subsample": [.8],
         "colsample_bytree": [1], "gamma": [0, 1, 5], "lambda": [.01, .1, 1]}), \
 \
       ("Adapative Boost", ada_boost, {"n_estimators": [100], "learning_rate": [.6, .8, 1]}), \
       ("Gradient Boost", grad_boost, {}), \
 \
       ("Histogram GB", hist_grad_boost,
        {"loss": ["binary_crossentropy"], "min_samples_leaf": [5, 10, 20, 40, 50], "l2_regularization": [0, .1, 1]})]

In [15]:
#創建stack_list儲存模型Train Score與Test Score
stack_list = []
#創建Evaluation儲存模型儲存Train Score, Test Score(Accuracy), Precision, Sensitivity, Specificity, F1_score
train_scores = pd.DataFrame(columns = ["Name", "Train Score", "Test Score(Accuracy)", "Precision", "Sensitivity", "Specificity", "F1_score"])

In [16]:
# 計算建立模型的時間(起點)
start = time.time()

In [17]:
#利用GridSearchCV自動優化參數
i = 0
for name, clf1, param_grid in clf:
    clf = GridSearchCV(clf1, param_grid=param_grid, scoring="accuracy", cv=sk_fold, return_train_score=True, n_jobs=7) #啟用7核心加速運算
    clf.fit(X_train, y_train.values.ravel())
    y_pred = clf.best_estimator_.predict(X_test)

    cm = confusion_matrix(y_test, y_pred)
    print(cm)
    print("=====================================")
    #儲存Train Score, Test Score(Accuracy), Precision, Sensitivity, Specificity, F1_score到train_scores dataframe中
    train_scores.loc[i] = [name, clf.best_score_, (cm[0, 0] + cm[1, 1]) / (cm[0, 0] + cm[0, 1] + cm[1, 0] + cm[1, 1]),
                               (cm[0, 0] / (cm[0, 0] + cm[0, 1])), (cm[0, 0] / (cm[0, 0] + cm[1, 1])),
                               (cm[1, 1] / (cm[1, 1] + cm[0, 1])),
                               ((2 * cm[0, 0]) / (2 * cm[0, 0] + cm[1, 0] + cm[0, 1]))]
    stack_list.append(clf.best_estimator_)
    i = i + 1

est = [("g_nb", stack_list[0]), \
       ("knn", stack_list[1]), \
       ("ran_for", stack_list[2]), \
       ("log_reg", stack_list[3]), \
       ("dec_tree", stack_list[4]), \
       ("XGBoost", stack_list[5]), \
       ("ada_boost", stack_list[6]), \
       ("grad_boost", stack_list[7]), \
       ("hist_grad_boost", stack_list[8])]

[[1837  232]
 [ 151  209]]
[[2054   15]
 [ 306   54]]
[[1983   86]
 [ 133  227]]
[[2069    0]
 [ 360    0]]
[[1914  155]
 [ 154  206]]
[22:52:15] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[[1976   93]
 [ 140  220]]
[[1962  107]
 [ 150  210]]
[[1966  103]
 [ 137  223]]
[[1980   89]
 [ 152  208]]


In [18]:
#集成學習
sc = StackingClassifier(estimators=est,final_estimator = None,cv=sk_fold,passthrough=False)
sc.fit(X_train,y_train)
y_pred = sc.predict(X_test)
cm1 = confusion_matrix(y_test,y_pred)
y_pred_train = sc.predict(X_train)
cm2 = confusion_matrix(y_train,y_pred_train)
stacking = pd.Series(["Stacking",(cm2[0,0]+cm2[1,1])/(cm2[0,0]+cm2[0,1]+cm2[1,0]+cm2[1,1]), (cm1[0, 0] + cm1[1, 1]) / (cm1[0, 0] + cm1[0, 1] + cm1[1, 0] + cm1[1, 1]),
                           (cm1[0, 0] / (cm1[0, 0] + cm1[0, 1])), (cm1[0, 0] / (cm1[0, 0] + cm1[1, 1])),
                           (cm1[1, 1] / (cm1[1, 1] + cm1[0, 1])), ((2 * cm1[0, 0]) / (2 * cm1[0, 0] + cm1[1, 0] + cm1[0, 1]))],
              index=["Name", "Train Score", "Test Score(Accuracy)", "Precision", "Sensitivity", "Specificity", "F1_score"])
train_scores = train_scores.append(stacking, ignore_index=True)
print(train_scores)

[22:53:04] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[22:53:07] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[22:53:08] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[22:53:08] WARNING: C:/Users/Administrator/workspace/xgboost-win64_rel

In [19]:
#test資料集preprocessing
#顯示資料集
print(test)
#顯示資料筆數
print(test.shape)
#顯示資料統計資訊
print(test.describe())
#顯示欄位屬性
print(test.dtypes)

       0               1                        2              3   \
0      ID  Administrative  Administrative_Duration  Informational   
1    6162               0                        0              0   
2    8143               3                       33              0   
3    5571               0                        0              0   
4    3933               5                       79              1   
..    ...             ...                      ...            ...   
896  5887               2                      738              0   
897  5273               0                        0              2   
898  5833               0                        0              0   
899  2119               0                        0              0   
900  4448               1                        0              0   

                         4               5                        6   \
0    Informational_Duration  ProductRelated  ProductRelated_Duration   
1                         0

In [20]:
#檢查test是否具有遺漏值(未發現NA)
total_test = test.isnull().sum().sort_values(ascending=False)
percent_test = (test.isnull().sum()/test.isnull().count()).sort_values(ascending=False)
missing_data_test = pd.concat([total_test, percent_test], axis=1, keys=['Total', 'Percent'])
print(missing_data_test)

    Total  Percent
17      0      0.0
16      0      0.0
1       0      0.0
2       0      0.0
3       0      0.0
4       0      0.0
5       0      0.0
6       0      0.0
7       0      0.0
8       0      0.0
9       0      0.0
10      0      0.0
11      0      0.0
12      0      0.0
13      0      0.0
14      0      0.0
15      0      0.0
0       0      0.0


In [21]:
# 刪除第0列(去除欄位名稱)
test_new = test.drop([0])
print(test_new)

       0  1    2  3     4    5            6         7         8          9   \
1    6162  0    0  0     0   12        482.5      0.02      0.04          0   
2    8143  3   33  0     0   17   386.416667         0      0.01          0   
3    5571  0    0  0     0    9        154.8  0.018182  0.030303  20.749351   
4    3933  5   79  1    12   54  1603.333333  0.005085  0.025424          0   
5     934  2   17  1   105    9       148.25      0.02     0.022          0   
..    ... ..  ... ..   ...  ...          ...       ...       ...        ...   
896  5887  2  738  0     0   26  1840.666667     0.008     0.042          0   
897  5273  0    0  2  26.5  149  6001.171475     0.004  0.022011          0   
898  5833  0    0  0     0    2            0       0.2       0.2          0   
899  2119  0    0  0     0   15       263.25  0.013333      0.04          0   
900  4448  1    0  0     0    7           37  0.083333  0.108333          0   

      10 11 12  13 14  15 16 17  
1      0  8  3  1

In [22]:
#轉換資料型態為float
test_new = test_new.astype('float')

In [23]:
#顯示數據敘述統計
print(test_new.describe())

                0           1            2           3            4   \
count   900.000000  900.000000   900.000000  900.000000   900.000000   
mean   4491.621111    2.420000    79.843749    0.537778    33.816267   
std    2571.975086    3.271415   154.968236    1.323071   123.149610   
min       3.000000    0.000000     0.000000    0.000000     0.000000   
25%    2339.000000    0.000000     0.000000    0.000000     0.000000   
50%    4460.500000    1.000000    17.100000    0.000000     0.000000   
75%    6634.250000    4.000000    96.187500    0.000000     0.000000   
max    8978.000000   18.000000  1715.000000   16.000000  1312.400000   

               5             6           7           8           9   \
count  900.000000    900.000000  900.000000  900.000000  900.000000   
mean    33.022222   1225.327672    0.021056    0.041494    6.990629   
std     43.370427   1725.710370    0.047337    0.047904   20.153000   
min      0.000000      0.000000    0.000000    0.000000    0.000000

In [24]:
#選擇最佳模型，並輸出預測結果
#觀察Train Score, Test Score(Accuracy), Precision, Sensitivity, Specificity, F1_score後，選擇Random Forest作為預測模型
X_submit = test_new.iloc[:,1:]
stack_list[2].fit(train_new.iloc[:,1:18],train_new.iloc[:,18:])
y_submit = stack_list[2].predict(X_submit)

y_submit= pd.DataFrame(y_submit)
y_submit.index +=1

# FRAMING OUR SOLUTION
y_submit.columns = ['HasRevenue']
y_submit['ID'] = np.arange(1,y_submit.shape[0]+1)
y_submit = y_submit[['ID', 'HasRevenue']]

y_submit.to_csv(r'C:\Users\88691\PycharmProjects\hotel_predit\Submission.csv', index = False)

In [25]:
# 計算建立模型的時間(終點)
end = time.time()
spend = end - start
hour = spend // 3600
minu = (spend - 3600 * hour) // 60
sec = int(spend - 3600 * hour - 60 * minu)
print(f'一共花費了{hour}小時{minu}分鐘{sec}秒')

一共花費了0.0小時4.0分鐘31秒
